# Export trades ticks and immediately preceding book ticks using dedicated service - C#

### Overview
Here we want to extract trade ticks and also know about the state of the book immediately before each trade.

This sample is designed to request simultaneously trades and top of the book and illustrate market activity over a look back period.

### Inputs/outputs
Extraction sample requires instrument's identifier, date time intervals as per inputs and returns top fo the book the occured trades data points.

### Services used
This sample uses *gRPC requests* in order to retrieve trades and top of the book from the hosted service. The queried endpoint in this script are:
* *TickTradesAndBookService*: to directly retrieve trades and top of the book tick data from the server.

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    
***

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis, 2.32.*"


In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;

using TickTradesAndBookService = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookService;
using TickTradesAndBookRequest = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookRequest;
using TickTradesAndBookResponse = Systemathics.Apis.Services.Tick.V1.TickTradesAndBookResponse;
using Trade = Systemathics.Apis.Type.Shared.V1.Trade;
using Book = Systemathics.Apis.Type.Shared.V1.Book;
using BookUpdates = Systemathics.Apis.Type.Shared.V1.BookUpdates;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = new Grpc.Core.Metadata();
try { headers = TokenHelpers.GetTokenAsMetaData(); } catch {}

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data
To request *tick trades and book* service, we need to specify:
* Instrument identifier : the ticker and the exchange code (MIC) define a unique instrument
* Time period selection: select start and end dates and time intervals (Using UTC Time Zone)

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create time intervals
var start = new DateTime(2022, 01, 14);
var end = new DateTime(2022, 01, 14);
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = start.Year, Month = start.Month, Day = start.Day },
  EndDate = new Date { Year = end.Year, Month = end.Month, Day = end.Day }
};

// Build the market data request time interval (we are using Google date time format : we have to cast the dates)
// UTC time zone
var timeInterval = new TimeInterval()
{
     StartTime = new TimeOfDay { Hours = 14, Minutes = 00, Seconds = 00 },
     EndTime = new TimeOfDay { Hours = 15, Minutes = 00, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns the request reply:

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create requests for *trades* and *top of the book* data and instantiates the service:

In [ ]:
// Generate the tick trades and book request
var identifier = new IdentifierAndLevel { Exchange = exchange, Ticker = ticker, Level = Level.TradesAndBook };
var request = new TickTradesAndBookRequest{ 
    Constraints = constraints, 
    MaxDepth = 1, 
    BookUpdates = BookUpdates.SnapshotsOnly,
    Adjustment = false
};
request.Identifiers.Add(new [] {identifier}); 

#### 3.4 Export data
Tick by tick data is sent through a *stream*, the following code snippet iterates ticks and keep trades ticks and their immediately preceding book tick (if any)

Output the results to CSV


In [ ]:
using System;
using System.Linq;
using System.IO;

var d = new DirectoryInfo("output"); if (!d.Exists) { d.Create(); }
var filename = $"output/{ticker}-{exchange}_trades_and_immediately_preceding_bbo.csv";

var csv_lines_count = 0;
var csv_book_lines_count = 0;
var csv_trade_lines_count = 0;

TickTradesAndBookResponse previous = null;
var not_shown_book_ticks_before = 0;
var total_ticks = 0;
var total_trade_ticks = 0;
var total_book_ticks = 0;
var last_is_trade = false;

using(var sw = File.CreateText(filename))
{
    sw.WriteLine($"time_stamp,seconds,nanos,bid_size,bid_price,ask_price,ask_size,trade_price,trade_size,not_shown_book_ticks_before");

    var service = new TickTradesAndBookService.TickTradesAndBookServiceClient(channel);
    var response = service.TickTradesAndBook(request, headers);
    await foreach (var current in response.ResponseStream.ReadAllAsync())
    {
        if(current?.Data == null)
        {
            // Skip the mapping data
            continue;
        }

        if (current.Data.Trade != null)
        {
            total_ticks += 1;
            total_trade_ticks += 1;
        }
                    
        if (current.Data.Book != null)
        {
            total_ticks += 1;
            total_book_ticks += 1;
        }

        if (current.Data.Trade != null) 
        {      
            if (previous?.Data?.Book != null)
            {
                var seconds = previous.Data.TimeStamp.Seconds;
                var nanos = previous.Data.TimeStamp.Nanos;
                var book_time = previous.Data.TimeStamp.ToDateTime();
                var bid_size = previous.Data.Book.Bid[0].Size;
                var bid_price = previous.Data.Book.Bid[0].Price;
                var ask_price = previous.Data.Book.Ask[0].Price;
                var ask_size = previous.Data.Book.Ask[0].Size;
                csv_lines_count += 1;
                csv_book_lines_count += 1;
                var s = not_shown_book_ticks_before > 0 ? $"{not_shown_book_ticks_before}" : string.Empty;
                var b = $"{book_time:O}".ToString().Replace("T"," ").Replace("Z", string.Empty).TrimEnd(new [] { '0' });
                sw.WriteLine($"{b},{seconds},{nanos},{bid_size},{bid_price},{ask_price},{ask_size},,,{s}");
            }

            var seconds2 = current.Data.TimeStamp.Seconds;
            var nanos2 = current.Data.TimeStamp.Nanos;
            var trade_time = current.Data.TimeStamp.ToDateTime();
            var trade_price = current.Data.Trade.Price;
            var trade_size = current.Data.Trade.Size;
            csv_lines_count += 1;
            csv_trade_lines_count += 1;
            var t = $"{trade_time:O}".ToString().Replace("T"," ").Replace("Z", string.Empty).TrimEnd(new [] { '0' });
            sw.WriteLine($"{t},{seconds2},{nanos2},,,,,{trade_price},{trade_size},");
            last_is_trade = true;
            not_shown_book_ticks_before = 0;
        }
        else
        {
            last_is_trade = false;
            not_shown_book_ticks_before += 1;
        }
    
        previous = current;
    }
}

Console.WriteLine($"#total_ticks={total_ticks:N0} #total_book_ticks={total_book_ticks:N0} #total_trade_ticks={total_trade_ticks:N0}");
Console.WriteLine($"#csv_lines_count={csv_lines_count:N0} #csv_book_lines_count={csv_book_lines_count:N0} #csv_trade_lines_count={csv_trade_lines_count:N0} [{filename}]");